# Power BI Connection Test - Interactive Authentication
Uses browser-based authentication to support MFA.

**Dataset Details:**
- Workspace ID: `bcb084ed-f8c9-422c-b148-29839c0f9227`
- Semantic Model ID: `a17de62e-8dc0-4a8a-acaa-2a9954de8c75`

In [0]:
%pip install azure-identity requests pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.3/191.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-94f3a408-c949-41c7-a53f-be5af906856c
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from azure.identity import InteractiveBrowserCredential, DeviceCodeCredential
import requests
import pandas as pd

# Configuration
TENANT_ID = "9f37a392-f0ae-4280-9796-f1864a10effc"  # Your tenant ID
CLIENT_ID = "1950a258-227b-4e31-a9cf-717495945fc2"  # Power BI public client

# Dataset information
WORKSPACE_ID = "bcb084ed-f8c9-422c-b148-29839c0f9227"
SEMANTIC_MODEL_ID = "a17de62e-8dc0-4a8a-acaa-2a9954de8c75"

print("Configuration loaded")

Configuration loaded


## Method 2: Device Code Flow (Best for Databricks/remote environments)

In [0]:
def generate_token_device_code(tenant_id: str, client_id: str) -> str:
    """
    Generate token using device code flow.
    You'll get a code to enter at microsoft.com/devicelogin
    """
    try:
        credential = DeviceCodeCredential(
            client_id=client_id,
            tenant_id=tenant_id,
        )
        
        # Get token for Power BI API
        print("\n⚠️  Follow the instructions above to authenticate")
        token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
        print("✓ Token generated successfully")
        return token.token
    except Exception as e:
        print(f"✗ Token generation failed: {str(e)}")
        raise

# Uncomment to use device code flow instead
access_token = generate_token_device_code(TENANT_ID, CLIENT_ID)
print(f"Token length: {len(access_token)} characters")


⚠️  Follow the instructions above to authenticate
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BHFEVBK7L to authenticate.
✓ Token generated successfully
Token length: 2048 characters


## Test Connection - Get Dataset Metadata

In [0]:
def get_dataset_info(token: str, dataset_id: str) -> dict:
    """
    Get metadata about the dataset to verify connection.
    """
    url = f"https://api.powerbi.com/v1.0/myorg/datasets/{dataset_id}"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    response = requests.get(url, headers=headers)
    print(f"Response status: {response.status_code}")
    
    if response.status_code == 200:
        print("✓ Successfully connected to dataset")
        return response.json()
    else:
        print(f"✗ Failed to connect: {response.text}")
        return {}

# Get dataset info
dataset_info = get_dataset_info(access_token, SEMANTIC_MODEL_ID)
if dataset_info:
    print(f"\nDataset Name: {dataset_info.get('name', 'N/A')}")
    print(f"Dataset ID: {dataset_info.get('id', 'N/A')}")
    print(f"Is Refreshable: {dataset_info.get('isRefreshable', 'N/A')}")

Response status: 200
✓ Successfully connected to dataset

Dataset Name: test_pbi
Dataset ID: a17de62e-8dc0-4a8a-acaa-2a9954de8c75
Is Refreshable: True


## Execute DAX Query

In [0]:
def execute_dax_query(token: str, dataset_id: str, dax_query: str) -> pd.DataFrame:
    """
    Execute a DAX query against the Power BI dataset.
    """
    url = f"https://api.powerbi.com/v1.0/myorg/datasets/{dataset_id}/executeQueries"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    body = {
        "queries": [
            {
                "query": dax_query
            }
        ],
        "serializerSettings": {
            "includeNulls": True
        }
    }
    
    print(f"Executing DAX query...")
    print(f"Query: {dax_query[:100]}...")
    response = requests.post(url, headers=headers, json=body, timeout=30)
    print(f"Response status: {response.status_code}")
    
    if response.status_code == 200:
        results = response.json().get("results", [])
        if results and results[0].get("tables"):
            rows = results[0]["tables"][0].get("rows", [])
            if rows:
                df = pd.DataFrame(rows)
                print(f"✓ Query successful: {len(df)} rows returned")
                return df
            else:
                print("⚠ Query returned no rows")
                return pd.DataFrame()
        else:
            print("⚠ No tables in response")
            return pd.DataFrame()
    else:
        print(f"✗ Query failed: {response.text}")
        return pd.DataFrame()

## Query Your Test Data
Replace `test_data` with the actual table name from the results above

In [0]:
# Simple query to get all data
# Replace 'test_data' with your actual table name
query = """
EVALUATE
TOPN(
    100,
    TestData
)
"""

df_result = execute_dax_query(access_token, SEMANTIC_MODEL_ID, query)
if not df_result.empty:
    print("\nQuery Results:")
    display(df_result.head(100))
    print(f"\nTotal rows: {len(df_result)}")
    print(f"Columns: {list(df_result.columns)}")

Executing DAX query...
Query: 
EVALUATE
TOPN(
    100,
    TestData
)
...
Response status: 200
✓ Query successful: 100 rows returned

Query Results:


TestData[fiscper],TestData[country],TestData[product],TestData[nsr],TestData[cogs],TestData[net_income]
2025001,US,product_a,948098.35,550339.99,284132.16
2025001,US,product_b,814986.7,484678.3,221491.96
2025001,US,product_c,140653.39,64144.59,58286.37
2025001,CH,product_a,750605.98,348458.51,318911.28
2025001,CH,product_b,362438.24,146104.42,172259.95
2025001,CH,product_c,511223.14,229828.67,209732.01
2025001,JP,product_a,828814.79,341783.11,329301.18
2025001,JP,product_b,297355.43,159337.8,84615.56
2025001,JP,product_c,347502.0,148410.79,132671.56
2025002,US,product_a,529548.09,245267.82,205204.95



Total rows: 100
Columns: ['TestData[fiscper]', 'TestData[country]', 'TestData[product]', 'TestData[nsr]', 'TestData[cogs]', 'TestData[net_income]']
